In [1]:
from datetime import date

today = date.today()
print("Today's date is:", today)
# (base) nana ~ % conda activate tensorflow
# Install a pip package in the current Jupyter kernel
# !python --version
# !pip install matplotlib

Today's date is: 2021-08-23


In [2]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import layers # 1.14.0
from tensorflow.keras.models import Model, load_model
import tensorflow.keras.backend as K

# %pylab inline
import random


from glob import glob  # directory paths
import pandas as pd    # big data management 
# from sklearn.cluster import KMeans  # ML package, clustering code, similarity measure
import numpy as np # 1.16.4
from numpy import array, zeros, prod
# import seaborn as sns
# print(tf.__version__)

In [3]:
import numpy # Previously it was 1.21 which got conflict with LSTM! I downgraded numpy version!
numpy.version.version

'1.19.5'

# Data Processing

In [4]:
# loads all the touch files into numpy format
dirs = glob('ExperimentSmall/*/') 

length = 250  # length of data file (10 seconds at 25 samples per second)
total_type = []  # sentence in survey corresponding to the touch data 
total_touch = []  # all touch data here
type_num = 33  # number of questions in survey

for path in dirs:
    files_path = glob(path + '*') # num_paths -> files_path
    for num_path in files_path:
        numstr = num_path.split('.')[-1]#Split the num_path from files_path until '.' from the end of the num_path
        if numstr.isdigit():
            num = int(numstr) 
                        
        temp_data = []
        with open(num_path, 'r') as f:  
            for _, line in enumerate(f):
                temp_data.append(line.split()[0].split(','))   # comma separated data
#                 print("temp_data:", temp_data) #['0', '169', '0', '86', '0', '0'],
#                 print(temp_data[-1])
        # if number of rows is less than length=250 repeat the last row to get 250 row in total
        while len(temp_data) < length:
            last_data = temp_data[-1]
#             print(last_data) # ['0', '323', '0', '0', '0', '0'] \\ ['0', '367', '0', '0', '0', '0']
            temp_data.append(last_data)
#         print("len(temp_data):", len(temp_data))
        for i in range(len(temp_data)-length): #len(temp_data)= 1672   1175   
            total_touch.append(temp_data[i:i+250])
            total_type.append(num) # 1, 2
#             print("total_type:", total_type)
            
#         print(total_type)
#         print(array(total_touch).shape, num_path) # which files cause error in shape 
# & convert it from 3d to 2d

total_touch = np.array(total_touch)
print("total_touch.shape:", total_touch.shape) 
# print("num is:", total_type)
total_length = len(total_touch) 
print("total_length:", total_length) 


total_touch.shape: (35517, 250, 6)
total_length: 35517


In [5]:
liwc_csv = pd.read_csv('liwc.csv')  # reading in the LIWC scores for all lyrics
print(len(liwc_csv))
liwc_csv[0:3]

310543


,A,B,C,WC,Analytic,Clout,Authentic,Tone,posemo,negemo,anx,anger,sad,female,male,tentat,certain,sexual,swear
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd...",32,11.24,73.40,7.84,99.00,12.50,0.0,0.0,0.0,0.0,9.38,3.12,6.25,0.00,0.0,0.0
1,ABBA,Ahe's My Kind Of Girl,"She's just my kind of girl, she makes me feel ...",42,1.00,31.71,2.82,70.57,2.38,0.0,0.0,0.0,0.0,19.05,0.00,7.14,4.76,0.0,0.0
2,ABBA,Ahe's My Kind Of Girl,And when we go for a walk in the park \nAnd s...,33,39.45,96.52,85.21,25.77,0.00,0.0,0.0,0.0,0.0,3.03,0.00,0.00,3.03,0.0,0.0


In [6]:
%%time
from __future__ import print_function 
csv_len = len(liwc_csv) # 310543
feature_num = len(liwc_csv.iloc[0][8:].values) # = 11 Identifies the number of features - 0:8 is meta-data
total_features = zeros((type_num, feature_num))
features = []

for i in range(csv_len):  # range over lyrics
    if i % 1000 == 0:
        print("i % 1000 == 0:", i / csv_len, end="\r")
    features.append(liwc_csv.iloc[i, 8:].values)  # associate feature 1 with its score in lyric 1
features = array(features)

# sentences in the experiment
text = ['Something that you said burnin\' in my head',
        'Y\'all want a single say fuck that  \nFuck that, fuck that  \nFuck that',
        'Naked - naked - just get back to basics  \nNaked - face it - you can\'t fake it when youre  \n',
        'All wound up  \nOn the edge',
        'I\'ve never felt good  \nI\'ve never felt bad',
        'Shell shock battle fatigue',
        'Good morning beautiful, a beautiful day',
        'The Humpty Dance is your chance to do the Hump',
        'Tears of rage, tears of pain',
        'Welcome to the pleasuredome',
        'But maybe I\'m crazy',
        'Still I\'m sad  \n',
        'Promise me love, promise me  \nKisses sweet love',
        'Cry baby cry  \nMake your mother sigh  \nShe\'s old enough to know better.  \nSo cry baby cry',
        'Lover, lover, lover  \n',
        'Never give up  \nNever hold you head down',
        'She\'s losing,',
        'Bitch,  \nStupid ass bitch',
        'If I, if I, if I, if I, if I, if I, if I, if I',
        'Damn  \nDamn girl',
        'Kill tonight, kill tonight  \n',
        'But try to find  \n',
        'Tell me why do my tears fall like rain\?  \nWorry worry worry',
        'I\'m drowning  \nAnger my bitter breath',
        'Sorry \(I\'m sorry\)',
        'Wake up in the morning, my hands cold in fear.',
        'All together now  \n\(All together now\)  \n',
        'Her feelings she hides  \n',
        'Boys boys boys  \nWe like boys in cars  \nBoys boys boys  \n',
        'He\'s a woman, she\'s a man  \n',
        'Mother\'s pride  \nBaby boy',
        'Strong girl, she\'s like an ocean',
        'He drank Coca-Cola, he was eating Wonder Bread',
        ]

for i, txt in enumerate(text):  # retrieve feature score for all 33 lyrics
    print(i, end='\r')
    total_features[i] = liwc_csv[liwc_csv['C'].str.contains(txt)].iloc[0, 8:].values
    
print("total_features:", total_features.shape)

total_features: (33, 11)498797268354 0.0450823235429554 0.7696196661975958
CPU times: user 20 s, sys: 307 ms, total: 20.3 s
Wall time: 20.3 s


In [7]:
features.shape

(310543, 11)

# LSTM Nana

In [8]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential


In [9]:
# define model
# (25, 6) provide sequence of 25 by 6 inputs

input_touch = layers.Input((250, 6))
# x = layers.LSTM(100, return_sequences = True)
x = layers.LSTM(200, return_sequences=True, recurrent_dropout=0.2)(input_touch)
x = layers.Dropout(0.2)(x)
x = layers.LSTM(200, recurrent_dropout=0.2)(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(feature_num, activation='relu')(x)

model = Model(input_touch, x)
model.compile(optimizer='RMSprop', loss='mse') #, metrics=['acc']
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 250, 6)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 250, 200)          165600    
_________________________________________________________________
dropout (Dropout)            (None, 250, 200)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               25728     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0     

In [10]:
# # define model
# model = Sequential()
# model.add(LSTM(50, activation='relu', input_shape=(99, 9)))
# model.add(Dense(1))
# model.compile(optimizer='adam', loss='mse')

In [11]:
import numpy as np
import tensorflow.keras.models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import random

In [12]:
import logging
import tensorflow
tensorflow.autograph.set_verbosity(0)
logging.getLogger("tensorflow").setLevel(logging.ERROR)


In [13]:
# Used to give an error of arange is not defined and kernel dies! 
def gen(batch_size, arr):
    i = 0
    while True:
        temp_arr = arr[i * batch_size:(i + 1) * batch_size]
        length = len(temp_arr)
        temp_touch = zeros((length, 250, 6))
        temp_type = zeros((length, feature_num))
        for j in range(length):
            temp_num = temp_arr[j]
            temp_touch[j] = total_touch[temp_num]
            temp_type[j] = total_features[total_type[temp_num]]
        i += 1
        if i * batch_size >= len(arr):
            i = 0
        yield temp_touch, temp_type
        
batch_size = 512 # 128 bud ba 222 steps_per_epoch
this_arr = arange(total_length)
random.shuffle(this_arr)
split = total_length // 10
validation_arr = this_arr[8 * split:9 * split]
test_arr = this_arr[9 * split:]
train_arr = this_arr[:8 * split]



In [14]:
# Run this cell if you want to decrease the data for training 
train_arr = train_arr[0:10000]
test_arr = test_arr[0:1500]
len(train_arr), len(test_arr)

(10000, 1500)

In [15]:
# https://www.geeksforgeeks.org/keras-fit-and-keras-fit_generator/
history_lstm = model.fit_generator(gen(batch_size, train_arr),
                              steps_per_epoch=int(len(train_arr) / batch_size) + 1,
                              epochs=10,#50
                              validation_data=gen(batch_size, validation_arr),
                              validation_steps=int(len(validation_arr) / batch_size) + 1,
                              )


Epoch 1/10


/Users/nana/miniforge3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-08-23 17:56:44.216042: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-23 17:56:44.216216: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


20/20 [==============================] - 637s 33s/step - loss: 192.9980 - val_loss: 141.0552
Epoch 2/10
20/20 [==============================] - 717s 36s/step - loss: 143.2214 - val_loss: 138.2721
Epoch 3/10
20/20 [==============================] - 751s 36s/step - loss: 138.3275 - val_loss: 136.1508
Epoch 4/10
20/20 [==============================] - 715s 35s/step - loss: 132.3651 - val_loss: 134.1617
Epoch 5/10
20/20 [==============================] - 752s 37s/step - loss: 123.7979 - val_loss: 132.9372
Epoch 6/10
20/20 [==============================] - 688s 34s/step - loss: 117.4327 - val_loss: 138.6464
Epoch 7/10
20/20 [==============================] - 722s 35s/step - loss: 109.9961 - val_loss: 130.2927
Epoch 8/10
20/20 [==============================] - 764s 37s/step - loss: 103.5910 - val_loss: 132.1920
Epoch 9/10
20/20 [==============================] - 941s 47s/step - loss: 98.6237 - val_loss: 121.0914
Epoch 10/10
20/20 [==============================] - 758s 37s/step - loss: 8

In [ ]:
# https://www.geeksforgeeks.org/keras-fit-and-keras-fit_generator/
history_lstm = model.fit_generator(gen(batch_size, train_arr),
                              steps_per_epoch=int(len(train_arr) / batch_size) + 1,
                              epochs=10,#50
                              validation_data=gen(batch_size, validation_arr),
                              validation_steps=int(len(validation_arr) / batch_size) + 1,
                              )


Epoch 1/10
20/20 [==============================] - 660s 33s/step - loss: 182.3389 - val_loss: 140.8456
Epoch 2/10
20/20 [==============================] - 652s 32s/step - loss: 140.1224 - val_loss: 139.6777
Epoch 3/10
20/20 [==============================] - 941s 47s/step - loss: 136.5582 - val_loss: 140.0770
Epoch 4/10
20/20 [==============================] - 686s 33s/step - loss: 131.8049 - val_loss: 138.5826
Epoch 5/10
20/20 [==============================] - 722s 35s/step - loss: 125.7657 - val_loss: 139.6485
Epoch 6/10
20/20 [==============================] - 682s 33s/step - loss: 120.3577 - val_loss: 137.0075
Epoch 7/10
 3/20 [===>..........................] - ETA: 10:45 - loss: 117.8415

In [ ]:
# import matplotlib.pyplot as plt
# # Plot training & validation loss values
# plt.plot(history_lstm.history['loss'][2:])
# plt.plot(history_lstm.history['val_loss'][2:])
# plt.title('Model_loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Valid'], loc='upper left')
# plt.grid(True)
# # plt.xticks(np.arange(1, 100, 5))
# plt.show()